In [133]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import json
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
# import matplotlib.pyplot as plt
# %matplotlib inline

'en_US'

In [189]:
with open('web/data/incomeTax.json') as data_file:    
    data = json.load(data_file)

# Testing
print data[0]["data"]["New Brunswick"]["brackets"][0]["rate"]
print data[0]["data"]["New Brunswick"]["brackets"][0]["upper"]

9.68
32730


In [190]:
def getAvgRate(location, income, data):
    
    def govtTaxAmount(govt):
        # Fetch the tax bracket data
        brackets = data[govt]["brackets"]
        untaxed_income = income # This will keep track of the part of the income that has not been taxed yet
        tax_paid = 0.0 # A running total of the amount of tax paid
        lower = 0 # The lower value of tax bracket
        n = 0 # Keeps track of which tax bracket we're in
        while untaxed_income > 0:
            # Fetch the current bracket
            bracket = brackets[n]
            # This checks to see if we're on the final bracket
            try:
                upper = int(bracket["upper"]) # Upper value of tax bracket
                rate = float(bracket["rate"])/100.0 # Tax rate of bracket
            except:
                upper = "max" # There is no upper value for the final bracket
                rate = float(bracket["rate"])/100.0

            if upper == "max" or income < upper:
                # Does the calculation for the final bracket the person falls into
                tax_paid = tax_paid + untaxed_income*rate
                untaxed_income = 0
            else:
                # Gets the tax paid on the brackets a person is totally above
                tax_paid = tax_paid + (upper - lower)*rate
                untaxed_income = untaxed_income - (upper - lower)
                lower = upper
                n = n + 1
        # This is the average rate the person ends up paying
        # Quebec abatement adjustment: http://www.fin.gc.ca/fedprov/altpay-eng.asp
        if (location == "Quebec" and govt == "Federal"):
            tax_paid = tax_paid*0.835
        return tax_paid
    
    # Get the base amount of tax that you pay
    fed_tax = govtTaxAmount("Federal")
    prov_tax = govtTaxAmount(location)
    
    # A function for deducting the personal exemption amount federally and provincially
    def govtCredits(govt):
        credit = data[govt]["credits"][0]
        credit_amount = int(credit["amount"])*float(credit["percent"])/100.0
        # Quebec abatement adjustment: http://www.fin.gc.ca/fedprov/altpay-eng.asp
        if (location == "Quebec" and govt == "Federal"):
            credit_amount = credit_amount*0.835
        return credit_amount
    
    # Subtract the tax credits
    fed_tax = fed_tax - govtCredits("Federal")
    prov_tax = prov_tax - govtCredits(location)
    
    # If you have any surtaxes to pay, this takes that into account
    try:
        # Get the surtaxes. Throws an error if there are none.
        surtaxes = data[location]["surtaxes"]
        prov_tax_with_surtax = prov_tax
        for surtax in surtaxes:
            lower_limit = int(surtax["lower"])
            rate = float(surtax["rate"])/100.0
            # If the amount of tax you pay is above the lower limit for each surtax,
            # add the amont of surtax you have to pay to the running total
            if prov_tax_after_credit > lower_limit:
                prov_tax_with_surtax += (prov_tax_after_credit - lower_limit) * rate
        prov_tax = prov_tax_with_surtax
    except:
        pass
    
    # Adds the amount for the health fee that some provinces have
    try:
        health_fee = 0
        # Get the health fee, or throw an error if there are none
        health_brackets = data[location]["healthfee"]
        for bracket in health_brackets:
            lower = int(bracket["lower"])
            upper = int(bracket["upper"])
            value = int(bracket["value"])
            rate = float(bracket["rate"])
            if (income >= lower) and (income <= upper):
                health_fee = health_fee + value
                health_fee = health_fee + (income - lower - 1) * rate / 100.0
                prov_tax = prov_tax + health_fee
    except:
        pass
    
    # This is for the special health fee that quebec has
    try:
        health_fee = 0
        # Get the health fee, or throw an error if there are none
        health_brackets = data[location]["healthfeeqc"]
        for bracket in health_brackets:
            lower = int(bracket["lower"])
            upper = int(bracket["upper"])
            value = float(bracket["value"])
            rate = float(bracket["rate"])
            max_amount = float(bracket["max"])
            if (income >= lower) and (income <= upper):
                health_fee = health_fee + value
                health_fee = health_fee + (income - lower - 1) * rate / 100.0
                # Limit the tax to the upper bound
                if (health_fee > max_amount):
                    health_fee = max_amount
                prov_tax = prov_tax + health_fee
    except:
        pass
    
    # Quebec has different rates for cpp and EI
    if (location != "Quebec"):
        cpp = data["Federal"]["cpp"]
        ei = data["Federal"]["ei"]
    else:
        cpp = data["Quebec"]["cpp"]
        ei = data["Quebec"]["ei"]
        
    # Adds the CPP tax
    max_income = int(cpp["maxIncome"])
    exemption = int(cpp["exemption"])
    rate = float(cpp["rate"])
    if (income > max_income):
        pen_income = max_income
    else:
        pen_income = income
    pen_income = pen_income - exemption
    cpp_amount = pen_income * rate / 100.0
    fed_tax = fed_tax + cpp_amount
    
    # Adds the EI tax
    max_income = int(ei["maxIncome"])
    rate = float(ei["rate"])

    if (income > max_income):
        ei_income = max_income
    else:
        ei_income = income
    ei_amount = ei_income * rate / 100.0
    fed_tax = fed_tax + ei_amount
    
    # Adds the Quebec Parental Insurance Plan
    if (location == "Quebec"):
        try:
            qpip = data["Quebec"]["qpip"]
            max_income = int(qpip["maxIncome"])
            rate = float(qpip["rate"])

            if (income > max_income):
                qpip_income = max_income
            else:
                qpip_income = income

            qpip_amount = qpip_income * rate / 100.0
            prov_tax = prov_tax + qpip_amount
        except:
            pass
    
    # Calculate the percentage tax, and return 0 if it is negative
    avg_fed_tax = max(round(fed_tax / income * 100, 2), 0)
    avg_prov_tax = max(round(prov_tax / income * 100, 2), 0)
    avg_tax_total = max(round(avg_prov_tax + avg_fed_tax, 2), 0)

    return {str(income) : {"avg_fed": avg_fed_tax, "avg_prov": avg_prov_tax, "avg_total": avg_tax_total}}

### Testing
getAvgRate("Quebec", 100000, data[0]["data"])

income = 80000
location = "Quebec"
test_data = data[0]["data"]

qpip = test_data["Quebec"]["qpip"]
max_income = int(qpip["maxIncome"])
rate = float(qpip["rate"])

if (income > max_income):
    qpip_income = max_income
else:
    qpip_income = income

qpip_amount = qpip_income * rate / 100.0
print qpip_amount

### Change the below to have more points for the first set of incomes

In [191]:
# Gets the average tax paid for a number of incomes for a given province in a given year
def provAvgs(province, year_data, year):
    incomeList = xrange(5000, 350000, 5000)
    return [getAvgRate(province, income, year_data) for income in incomeList]

In [193]:
def processYear(year_data, year):
    year_result = {}
    for province, brackets in year_data.iteritems():
        year_result[province] = provAvgs(province, year_data, year)
    return year_result

In [194]:
allData = {}
for year_record in data:
    year = year_record["year"]
    year_data = year_record["data"]
    allData[year] = processYear(year_data, year)
# print allData

In [205]:
# Convert the results into an object that can be visualized
# with each province representing a line, the income as the x-value
# and the avg tax as the y-value
def createYearDict(year):
    year_prov_avgs = allData[year]
    year_dict = {}
    for prov, income_data in year_prov_avgs.iteritems():
        prov_dict = {}
        for income_record in income_data:
            income = income_record.keys()[0]
            avg_tax = income_record.values()[0]['avg_total']
            prov_dict[int(income)] = avg_tax
        prov_series = Series(prov_dict)
        year_dict[prov] = prov_series
    year_df = pd.concat(year_dict, axis=1)
    year_df = year_df.drop('Federal', axis=1)
    return year_dict
createYearDict("2014")

{u'Alberta': 5000       0.00
 10000      3.39
 15000      9.54
 20000     13.72
 25000     17.34
 30000     19.75
 35000     21.48
 40000     22.77
 45000     23.95
 50000     25.38
 55000     26.21
 60000     26.69
 65000     27.09
 70000     27.45
 75000     27.75
 80000     28.02
 85000     28.25
 90000     28.55
 95000     28.95
 100000    29.30
 105000    29.62
 110000    29.90
 115000    30.17
 120000    30.42
 125000    30.64
 130000    30.84
 135000    31.03
 140000    31.29
 145000    31.55
 150000    31.80
           ...  
 200000    33.60
 205000    33.73
 210000    33.86
 215000    33.98
 220000    34.09
 225000    34.20
 230000    34.31
 235000    34.40
 240000    34.50
 245000    34.59
 250000    34.68
 255000    34.77
 260000    34.85
 265000    34.93
 270000    35.00
 275000    35.07
 280000    35.14
 285000    35.22
 290000    35.28
 295000    35.34
 300000    35.41
 305000    35.47
 310000    35.52
 315000    35.58
 320000    35.62
 325000    35.68
 330000    35.73
 3

In [224]:
# Convert the results into an object that can be visualized
# with each province representing a line, the income as the x-value
# and the avg tax as the y-value
def createYearDf(year):
    year_prov_avgs = allData[year]
    year_dict = {}
    for prov, income_data in year_prov_avgs.iteritems():
        prov_dict = {}
        for income_record in income_data:
            income = income_record.keys()[0]
            avg_tax = income_record.values()[0]['avg_total']
            prov_dict[int(income)] = avg_tax
        prov_series = Series(prov_dict)
        year_dict[prov] = prov_series
    # Convert the dictionary to a dataframe
    year_df = pd.concat(year_dict, axis=1)
    # Drop the Federal column since we're only interested in combined rates
    year_df = year_df.drop('Federal', axis=1)
    # Assign the year to a column to use in a dimensional model later
    year_df["Year"] = year
    # Get the income out of the index so that we can use it in a dimensional model later
    year_df = year_df.reset_index(level=0)
    year_df = year_df.rename(columns={"index": "Income"})
    return year_df

In [237]:
years = ["2005", "2014", "2015", "2016"]
array_of_dfs = []
for year in years:
    avg_rates_combined = createYearDf(year)
    df = pd.melt(avg_rates_combined, id_vars=["Year", "Income"], var_name="Province", value_name="Average Income Tax")
    array_of_dfs.append(df)
merged_data = pd.concat(array_of_dfs, ignore_index=True, axis=0)
merged_data.to_csv(path_or_buf="./MiscData/AvgRatesCombined.csv", index=False)

year_df.plot(figsize=(20, 12))

In [196]:
def convertYearD3(year_data):
    d3_year_data = []
    for province, avgs in year_data.iteritems():
        if province != 'Federal':
            line_data = {"values":[], "key": ""}
            line_data["key"] = province
            for income, avg in avgs.iteritems():
                line_data["values"].append({"x": income, "y": round(avg/100.0,3)})
            d3_year_data.append(line_data)
    return d3_year_data
# convertYearD3(createYearDict("2014"))

In [202]:
years = ["2005", "2014", "2015", "2016"]
all_d3_data = {"tax":{},"population":{}}
for year in years:
    single_year = createYearDict(year)
    single_year_d3 = convertYearD3(single_year)
    all_d3_data["tax"][year] = single_year_d3
all_d3_data

{'population': {},
 'tax': {'2005': [{'key': u'British Columbia',
    'values': [{'x': 5000, 'y': 0.0},
     {'x': 10000, 'y': 0.051},
     {'x': 15000, 'y': 0.127},
     {'x': 20000, 'y': 0.165},
     {'x': 25000, 'y': 0.188},
     {'x': 30000, 'y': 0.203},
     {'x': 35000, 'y': 0.216},
     {'x': 40000, 'y': 0.235},
     {'x': 45000, 'y': 0.245},
     {'x': 50000, 'y': 0.251},
     {'x': 55000, 'y': 0.257},
     {'x': 60000, 'y': 0.261},
     {'x': 65000, 'y': 0.265},
     {'x': 70000, 'y': 0.27},
     {'x': 75000, 'y': 0.276},
     {'x': 80000, 'y': 0.284},
     {'x': 85000, 'y': 0.29},
     {'x': 90000, 'y': 0.296},
     {'x': 95000, 'y': 0.302},
     {'x': 100000, 'y': 0.307},
     {'x': 105000, 'y': 0.312},
     {'x': 110000, 'y': 0.316},
     {'x': 115000, 'y': 0.32},
     {'x': 120000, 'y': 0.325},
     {'x': 125000, 'y': 0.329},
     {'x': 130000, 'y': 0.334},
     {'x': 135000, 'y': 0.337},
     {'x': 140000, 'y': 0.341},
     {'x': 145000, 'y': 0.344},
     {'x': 150000, 'y

In [201]:
with open('webcustom/avgTaxAll.json', 'w') as data_file:    
    json.dump(all_d3_data, data_file)

# Population Distribution

In [197]:
def commas(number):
    return locale.format("%d", number, grouping=True)

In [198]:
income_dist = pd.read_csv(filepath_or_buffer="./MiscData/IncDistCleaned.csv")
income_dist_dict = {}
income_dist["ActualTotal"] = 0
income_dist["PercentagePop"] = 0
income_dist["Label"] = ""
for (index, row) in income_dist.iterrows():
    
    # Get the total population for the percentage calculation
    prov = row["Province"]
    prov_data = income_dist.loc[income_dist["Province"] == prov,:]
    total_pop = prov_data.iloc[0]["Total"] + prov_data.iloc[1]["Total"]

    if row["Income"] == 0:
        income_dist.loc[index, "ActualTotal"] = row["Total"]
        income_dist.loc[index, "PercentagePop"] = 100.0 * row["Total"] / total_pop
        
        next_index = index + 1
        next_income = income_dist.loc[next_index, "Income"]
        income_dist.loc[index, "Label"] = "$" + str(row["Income"]) + " - " + str(commas(next_income - 1))
    elif row["Income"] == 250000:
        income_dist.loc[index, "ActualTotal"] = row["Total"]
        income_dist.loc[index, "PercentagePop"] = 100.0 * row["Total"] / total_pop
        
        income_dist.loc[index, "Label"] = "$250,000+"
    else:
        next_index = index + 1
        
        next_total = income_dist.loc[next_index, "Total"]
        income_dist.loc[index, "ActualTotal"] = row["Total"] - next_total
        income_dist.loc[index, "PercentagePop"] = 100.0 * (row["Total"] - next_total) / total_pop
        
        next_income = income_dist.loc[next_index, "Income"]
        income_dist.loc[index, "Label"] = "$" + str(commas(row["Income"])) + " - " + str(commas(next_income - 1))
income_dist.to_csv(path_or_buf="./MiscData/IncDistProcessed.csv")

In [199]:
row = income_dist.iloc[18]
prov = row["Province"]
prov_data = income_dist.loc[income_dist["Province"] == prov,:]
total_pop = prov_data.iloc[0]["Total"] + prov_data.iloc[1]["Total"]
100.0 * row["Total"] / total_pop

58.35221607903562